In [9]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_context("paper")
sns.set_style("whitegrid")
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams.update({'font.size': 20})



In [10]:
filename = "../../results_aggr/app_bootstrap_vm.csv"
insertsFigure = "appHistoryVmInserts.pdf"
queries1Figure = "appHistoryVmQueries.pdf"
queries2Figure = "appHistoryVmGroup.pdf"


In [11]:
df_values = pd.read_csv(filename, names=["commit", "type", "CI_min", "Median", "CI_max"], skiprows=1)
df_values.sort_values(by=["commit"], inplace=True)

In [12]:
for type in df_values.type.unique():
    print(f"Plotting type {type}")

    measurements = df_values.loc[(df_values['type'].str.startswith(type, na=False))]


    #Find % set y-limits
    min = -5
    max = 5
    for minSmall in measurements.CI_min.unique():
        if minSmall < min:
            min = minSmall

    for maxSmall in measurements.CI_max.unique():
        if maxSmall > max:
            max = maxSmall


    sns.set(style="whitegrid")
    #axes = plt.axes()

    plot, axes = plt.subplots(figsize=(15,9))
    axes.set_ylim([min, max])
    axes.set_xlim([0, 70])
    axes.set_xticks([0,10,20,30,40,50,60,70])

    plot = sns.lineplot(data=measurements, x="commit", y="CI_min", color="royalblue")
    plot = sns.lineplot(data=measurements, x="commit", y="CI_max", color="royalblue")

    x_commits = measurements.commit
    plot.fill_between(x=x_commits, y1=measurements.CI_min, y2=measurements.CI_max, facecolor = 'lightblue')
    plot = sns.lineplot(data=measurements, x="commit", y="Median", color="red")

    plot.set(ylim=(min, max))
    axes.legend(loc=4, fontsize="medium")
    plt.xlabel('Commit Number', fontsize=18)
    plt.ylabel('Relative Performance Change (%)', fontsize=18)

    if (type=="inserts"):
        plt.savefig(insertsFigure,bbox_inches='tight')
    if (type=="simple queries"):
        plt.savefig(queries1Figure,bbox_inches='tight')
    if (type=="group-by queries"):
        plt.savefig(queries2Figure,bbox_inches='tight')
    plt.clf()


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


Plotting type inserts
Plotting type simple queries
Plotting type group-by queries


<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>